In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import warnings
warnings.filterwarnings('ignore')

## Import & Prepare DataFrame

In [2]:
def remove_whitespaces(df: pd.DataFrame) -> None:
    """Remove whitespaces from column names and string values

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe to clean
    """
    # Remove whitespace from each column name
    df.columns = df.columns.str.strip()
    
    # Remove whitespace from each string value
    categorical_columns = df.select_dtypes("O").columns
    for column in categorical_columns:
        df[column] = df[column].str.strip()

In [3]:
MATCHES_DATA_PATH = "../data/matches_data/historical_matches.csv"
PLAYERS_DATA_PATH = "../data/players_data/players_all_prepared.csv"

# Load the matches data
matches = pd.read_csv(MATCHES_DATA_PATH)

# Load the players data
players = pd.read_csv(PLAYERS_DATA_PATH)


# Remove whitespaces from column names and string values
remove_whitespaces(matches)
remove_whitespaces(players)

In [4]:
# Matches date to datetime
matches["date"] = pd.to_datetime(matches["date"])
matches["year"] = matches["date"].dt.year

# Filter years in matches to match the given players data dates [2015, 2022]
matches = matches[matches["year"] >= 2014]

# Filter tournaments to keep only ones  having players as in world cup matches
to_keep_tournaments = ["Friendly", 
"AFC Asian Cup qualification",
"EAFF Championship",
"African Cup of Nations qualification",
"FIFA World Cup",
"Kirin Challenge Cup",
"UEFA Euro qualification",
"Superclásico de las Américas",
"Gulf Cup",
"AFC Asian Cup",
"African Cup of Nations",
"FIFA World Cup qualification",
"Copa América",
"Gold Cup",
"Copa América qualification",
"Kirin Cup",
"UEFA Euro",
"Confederations Cup",
"UEFA Nations League",
"CONCACAF Nations League qualification",
"CONCACAF Nations League",
"CONMEBOL–UEFA Cup of Champions"]

In [5]:
## Assert all to_keep torunments are in dataframe
for torn in to_keep_tournaments:
    all_tournaments = matches["tournament"].unique()
    found = matches[matches["tournament"] == torn].count().sum()
    assert found > 0, f"Zero mathces found for tournament {torn}"
    assert (all_tournaments == torn).sum() == 1, f"tournament {torn} NOT found in matches"


In [6]:
# Filter matches to keep only the ones in the tournaments to keep
matches = matches[matches["tournament"].isin(to_keep_tournaments)]

# matches net score
matches["net_score"] = matches["home_score"] - matches["away_score"]

# Keep only desired columns
matches = matches[["year", "home_team", "away_team", "net_score"]]

# Rename columns
matches.rename(columns={"home_team": "1st_team", "away_team": "2nd_team"}, inplace=True)

In [7]:
matches.reset_index(drop=True, inplace=True)
players.reset_index(drop=True, inplace=True)

## Merge Data

In [8]:
## Adjust year value. The FIFA data collected in 2023 represents the year 2022, and this applies to all the data.
## This adjusment is to correctly merge matches with the corresponding players data 
players["year"] -= 1


In [9]:
## get unique positions
all_positions = players["nation_position"].unique()
all_positions

array(['GK', 'RWB', 'LCB', 'SUB', 'LCM', 'RW', 'LW', 'RCM', 'LWB', 'RCB',
       'CB', 'ST', 'RES', 'LB', 'RDM', 'LM', 'RB', 'RM', 'LDM', 'CDM',
       'RS', 'CAM', 'LS', 'LF', 'RF', 'CF', 'CM', 'LAM', 'RAM'],
      dtype=object)

In [17]:
drop_ind = []
for i, row in matches.iterrows():
    year = row["year"]
    team1 = row["1st_team"].lower().strip()
    team2 = row["2nd_team"].lower().strip()

    # Check if team1 is in players data
    team1_players = players[(players["year"] == year) & (players["nationality_name"].str.lower() == team1)]
    if team1_players.empty:
        drop_ind.append(i)
        continue

    # Check if team2 is in players data 
    team2_players = players[(players["year"] == year) & (players["nationality_name"].str.lower()== team2)]
    if team2_players.empty:
        drop_ind.append(i)
        continue

In [18]:
# invalid matches ratio
len(drop_ind)/matches.shape[0]

0.8362068965517241

In [19]:
matches.drop(drop_ind, inplace=True)

In [20]:
grouped_players = players.groupby(["nationality_name", "year", "nation_position"]).mean()

In [28]:
pos+ "_"+plyrs_record.index + "_2nd"

Index(['RAM_height_cm_2nd', 'RAM_weight_kg_2nd', 'RAM_age_2nd',
       'RAM_overall_2nd', 'RAM_potential_2nd', 'RAM_attacking_crossing_2nd',
       'RAM_attacking_finishing_2nd', 'RAM_attacking_heading_accuracy_2nd',
       'RAM_attacking_short_passing_2nd', 'RAM_attacking_volleys_2nd',
       'RAM_skill_dribbling_2nd', 'RAM_skill_curve_2nd',
       'RAM_skill_fk_accuracy_2nd', 'RAM_skill_long_passing_2nd',
       'RAM_skill_ball_control_2nd', 'RAM_movement_acceleration_2nd',
       'RAM_movement_sprint_speed_2nd', 'RAM_movement_agility_2nd',
       'RAM_movement_reactions_2nd', 'RAM_movement_balance_2nd',
       'RAM_power_shot_power_2nd', 'RAM_power_jumping_2nd',
       'RAM_power_stamina_2nd', 'RAM_power_strength_2nd',
       'RAM_power_long_shots_2nd', 'RAM_mentality_aggression_2nd',
       'RAM_mentality_interceptions_2nd', 'RAM_mentality_positioning_2nd',
       'RAM_mentality_vision_2nd', 'RAM_mentality_penalties_2nd',
       'RAM_defending_marking_awareness_2nd',
       'RAM_de

In [32]:
grouped_players.loc["Argentina", 2014, "CF"].index

Index(['height_cm', 'weight_kg', 'age', 'overall', 'potential',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'defending_marking_awareness', 'defending_standing_tackle',
       'defending_sliding_tackle', 'goalkeeping_diving',
       'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning', 'goalkeeping_reflexes'],
      dtype='object')

In [36]:
plyrs_record.T.reset_index(drop=True)

,height_cm,weight_kg,age,overall,potential,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,...,mentality_vision,mentality_penalties,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,189.0,76.0,33.0,86.0,86.0,62.0,67.0,68.0,88.0,50.0,...,84.0,60.0,84.0,84.0,83.0,5.0,8.0,13.0,9.0,13.0


In [37]:
matches_copy = matches.copy()

,year,1st_team,2nd_team,net_score
5,2014,Norway,Poland,-3.0


In [65]:
pd.DataFrame(np.concatenate([matches_copy.iloc[0].T, plyrs_record.T]))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [67]:
tt = pd.concat([pd.DataFrame(matches_copy.iloc[0]).T.reset_index(drop=True), plyrs_record.T.reset_index(drop=True)], axis=1, ignore_index=True)

In [69]:
tt.columns = pd.DataFrame(matches_copy.iloc[0]).T.columns.tolist() + plyrs_record.T.columns.tolist()
tt

,year,1st_team,2nd_team,net_score,height_cm,weight_kg,age,overall,potential,attacking_crossing,...,mentality_vision,mentality_penalties,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,2014,Norway,Poland,-3.0,189.0,76.0,33.0,86.0,86.0,62.0,...,84.0,60.0,84.0,84.0,83.0,5.0,8.0,13.0,9.0,13.0


In [81]:
pd.DataFrame(grouped_players.loc["Argentina", 2014, "CF"])

,Argentina
,2014
,CF
height_cm,169.0
weight_kg,67.0
age,27.0
overall,93.0
potential,95.0
attacking_crossing,84.0
attacking_finishing,94.0
attacking_heading_accuracy,71.0


In [90]:
plyrs_record

,GK_height_cm_2nd,GK_weight_kg_2nd,GK_age_2nd,GK_overall_2nd,GK_potential_2nd,GK_attacking_crossing_2nd,GK_attacking_finishing_2nd,GK_attacking_heading_accuracy_2nd,GK_attacking_short_passing_2nd,GK_attacking_volleys_2nd,...,GK_mentality_vision_2nd,GK_mentality_penalties_2nd,GK_defending_marking_awareness_2nd,GK_defending_standing_tackle_2nd,GK_defending_sliding_tackle_2nd,GK_goalkeeping_diving_2nd,GK_goalkeeping_handling_2nd,GK_goalkeeping_kicking_2nd,GK_goalkeeping_positioning_2nd,GK_goalkeeping_reflexes_2nd
0,190.0,89.0,25.0,85.0,88.0,16.0,14.0,18.0,41.0,17.0,...,48.0,24.0,11.0,18.0,12.0,84.0,81.0,79.0,87.0,86.0


In [93]:
pd.DataFrame(match).T.reset_index(drop=True)

,year,1st_team,2nd_team,net_score
0,2022,Portugal,Spain,-1.0


In [94]:
pd.concat([pd.DataFrame(match).T.reset_index(drop=True), plyrs_record], axis=1, ignore_index=True)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,2022,Portugal,Spain,-1.0,190.0,89.0,25.0,85.0,88.0,16.0,...,48.0,24.0,11.0,18.0,12.0,84.0,81.0,79.0,87.0,86.0


In [101]:
dataset = pd.DataFrame()
error_count = 0
counter = 0
for _, match in matches.iterrows():
    for pos in all_positions:
        # new_row = pd.DataFrame()
        second_team =  match["2nd_team"]
        year = match["year"]
        counter+=1
        try:
            plyrs_record = pd.DataFrame(grouped_players.loc[second_team, year, pos])
            plyrs_record = plyrs_record.T.reset_index(drop=True)
            plyrs_record.columns = pos+ "_"+plyrs_record.columns+ "_2nd"
            new_row = pd.concat([pd.DataFrame(match).T.reset_index(drop=True), plyrs_record.reset_index(drop=True)], axis=1, ignore_index=True)
            new_row.columns = pd.DataFrame(match).T.columns.tolist() + plyrs_record.columns.tolist()
            # print(new_row)
            dataset = pd.concat([dataset, new_row], axis=0, ignore_index=True)
        except Exception as e:
            error_count+=1
            # print(f"Error --> {e}" )

        
        
    

In [105]:
dataset.dropna(axis=1, how="all")

,year,1st_team,2nd_team,net_score,GK_height_cm_2nd,GK_weight_kg_2nd,GK_age_2nd,GK_overall_2nd,GK_potential_2nd,GK_attacking_crossing_2nd,...,RES_mentality_vision_2nd,RES_mentality_penalties_2nd,RES_defending_marking_awareness_2nd,RES_defending_standing_tackle_2nd,RES_defending_sliding_tackle_2nd,RES_goalkeeping_diving_2nd,RES_goalkeeping_handling_2nd,RES_goalkeeping_kicking_2nd,RES_goalkeeping_positioning_2nd,RES_goalkeeping_reflexes_2nd
0,2014,Norway,Poland,-3.0,196.0,84.0,24.0,80.0,83.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,Norway,Poland,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,Norway,Poland,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,Norway,Poland,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,Norway,Poland,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13429,2022,Portugal,Spain,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13430,2022,Portugal,Spain,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,63.666667,50.0,46.0,36.666667,35.333333,32.333333,32.666667,34.0,33.0,34.0
13431,2022,Portugal,Spain,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13432,2022,Portugal,Spain,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
error_count

175342

In [10]:
data = players_2.merge( matches, left_on="nationality_name", right_on="away_team")
# data = players_1.merge(data, left_on="nationality_name", right_on="home_team")

In [11]:
data

,long_name,height_cm,weight_kg,age,overall,potential,sofifa_id,nation_position,nationality_name,attacking_crossing,...,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year_y
0,Saad Al Sheeb,188,79,32,70,70,180548.0,GK,Qatar,11,...,2014-05-30,North Macedonia,Qatar,0.0,0.0,Friendly,Rieti,Italy,True,2014
1,Saad Al Sheeb,188,79,32,70,70,180548.0,GK,Qatar,11,...,2014-09-03,Morocco,Qatar,0.0,0.0,Friendly,Rabat,Morocco,False,2014
2,Saad Al Sheeb,188,79,32,70,70,180548.0,GK,Qatar,11,...,2014-11-13,Saudi Arabia,Qatar,1.0,1.0,Gulf Cup,Riyadh,Saudi Arabia,False,2014
3,Saad Al Sheeb,188,79,32,70,70,180548.0,GK,Qatar,11,...,2014-11-16,Yemen,Qatar,0.0,0.0,Gulf Cup,Riyadh,Saudi Arabia,True,2014
4,Saad Al Sheeb,188,79,32,70,70,180548.0,GK,Qatar,11,...,2014-11-19,Bahrain,Qatar,0.0,0.0,Gulf Cup,Riyadh,Saudi Arabia,True,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402712,Amrinder Singh,186,81,21,50,54,223760.0,SUB,India,25,...,2021-09-02,Nepal,India,1.0,1.0,Friendly,Kathmandu,Nepal,False,2021
402713,Amrinder Singh,186,81,21,50,54,223760.0,SUB,India,25,...,2021-09-05,Nepal,India,1.0,2.0,Friendly,Kathmandu,Nepal,False,2021
402714,Amrinder Singh,186,81,21,50,54,223760.0,SUB,India,25,...,2022-03-23,Bahrain,India,2.0,1.0,Friendly,Riffa,Bahrain,False,2022
402715,Amrinder Singh,186,81,21,50,54,223760.0,SUB,India,25,...,2022-03-26,Belarus,India,3.0,0.0,Friendly,Isa Town,Bahrain,True,2022


In [ ]:
temp_df.shape[0]/matches.shape[0]

0.21334418226200164